In [ ]:
import pandas as pd
import datasets

# helyi CSV fájl
input_data_file = ''

# CSV három oszloppal:
#    * question -- az elvárt felhasználói bemenet
#    * response_j -- a model elvárt válasza
#    * response_k -- példa egy rossz model válaszra
df = pd.read_csv(input_data_file)
# nem szabad, hogy None legyen az adatokban
df.dropna(inplace=True)

# HuggingFace dataset
ds = datasets.Dataset.from_pandas(df)

# dataset feltőltése s3-ba
s3_training_input_path = f's3://....'
ds.save_to_disk(s3_training_input_path)

# Train model

In [ ]:
from sagemaker.pytorch import PyTorch

hyperparameters = {
    'aws_region': '...',
    'huggingface_token': '...',
    'dataset_path': s3_training_input_path,
    'model_name': 'meta-llama/Llama-3.2-3B-Instruct'
}

pytorch_version = "2.2.0"
python_version = "py310"

pytorch_estimator = Pytorch(
    source_dir="./src",
    entry_point="launch.py",
    train_instance_type="ml.g5.12xlarge",
    train_instance_count=1,
    framework_version=pytorch_version,
    py_version=python_version,
    hyperparameters=hyperparameters
    output_path="s3://..."
    max_run=432000,
    # user settings
    role=role,
    subnets=["..."],
    security_group_ids=["..."],
    tags=[{"Key": "...", "Value": "..."}]
)

In [ ]:
s3_training_data_path = "s3://..."
pytorch_estimator.fit(s3_training_data_path)

# Deploy the model

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.huggingface import get_huggingface_llm_image_uri

llm_image = get_huggingface_llm_image_uri("huggingface")
llama_dpo_model = HuggingFaceModel(
    model_data=pytorch_estimator.model_data,
    role=role,
    image_url=llm_image
    # pytorch_version=pytorch_version,
    # py_version=python_version,
    # transformer_version="4.37.0",
    env={
        "HF_TASK": "text-generation",
        "HF_MODEL_ID": "/opt/ml/model"
    }
)

llama_dpo_model.vpc_config = {
    "Subnets": ["..."],
    "SecurityGroupIds": ["..."]
}

In [ ]:
endpoint_name = "llama-dpo"

predictor = llama_dpo_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.4xlarge",
    tags=[{"Key": "...", "Value": "..."}]
)

# Invoke endpoint

In [ ]:
import json
import boto3

payload = json.dumps({
    "inputs": "valami bemenet",
    "parameters": {
        "max_new_tokens": "...", 
        "top_p": "...", 
        "temperature": "...", 
        "return_full_text": False
    }
})

client = boto3.client("runtime.sagemaker")
response = client.invoke_endpoint(
    EndpointName=predictor.endpoint,
    ContentType="application/json",
    Body=payload,
    CustomAttributes="accpet_elu=true"
)

response = response.loads(response["Body"].read())
response